###Fuente de la data

Los micro-datos gestionados en este notebook son tomados de www.datos.gov.co y corresponden a datos cuya fuente primaria es la Fiscalía General de la Nación

In [1]:
import pandas as pd

###Archivo 6d52-qyqg.csv

Contiene el total de noticias criminales por delito registrados en el Sistema Penal Oral Acusatorio en la Ley 906 de 2004 y Ley 1098 de 2006 desde hechos ocurridos en 2010.

In [2]:
df = pd.read_csv("https://www.datos.gov.co/resource/6d52-qyqg.csv?$limit=2890487")
df.head(3)

,criminalidad,es_archivo,es_preclusion,estado,etapa_caso,ley,pais_hecho,departamento_hecho,municipio_hecho,seccional,a_o_hechos,a_o_entrada,a_o_denuncia,delito,grupo_delito,consumado,total_procesos
0,SI,SI,NO,INACTIVO,INDAGACIÓN,Ley 906,Colombia,Magdalena,SANTA MARTA,DIRECCIÓN SECCIONAL DE MAGDALENA,2013,2013,2013,HOMICIDIO ART. 103 C.P.,HOMICIDIO DOLOSO,NO,108
1,NO,SI,NO,INACTIVO,INDAGACIÓN,Ley 906,Colombia,Antioquia,MEDELLÍN,DIRECCIÓN SECCIONAL DE MEDELLÍN,2021,2021,2021,ESTAFA. ART. 246 C.P. MENOR CUANTIA,ESTAFA,NO APLICA,1771
2,SI,SI,NO,INACTIVO,INDAGACIÓN,Ley 906,Colombia,Cundinamarca,FUSAGASUGÁ,DIRECCIÓN SECCIONAL DE CUNDINAMARCA,2014,2014,2014,LESIONES PERSONALES CON INCAPACIDAD MENOR 60 DIAS,LESIONES PERSONALES,NO APLICA,19


###Revisión y limpieza para integrarlo a la base de datos

### - Resumen de la estructura del dataset

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2890487 entries, 0 to 2890486
Data columns (total 17 columns):
 #   Column              Dtype 
---  ------              ----- 
 0   criminalidad        object
 1   es_archivo          object
 2   es_preclusion       object
 3   estado              object
 4   etapa_caso          object
 5   ley                 object
 6   pais_hecho          object
 7   departamento_hecho  object
 8   municipio_hecho     object
 9   seccional           object
 10  a_o_hechos          int64 
 11  a_o_entrada         object
 12  a_o_denuncia        object
 13  delito              object
 14  grupo_delito        object
 15  consumado           object
 16  total_procesos      int64 
dtypes: int64(2), object(15)
memory usage: 374.9+ MB


###- Eliminar filas con noticias criminales que no hayan ocurrido en Colombia

In [4]:
crime_news_col = df[df['pais_hecho']=='Colombia']
len(crime_news_col)

2889896

### Eliminación de columnas irrelevantes para el proyecto

In [5]:
crime_news = crime_news_col.drop(columns=['es_archivo','es_preclusion','estado','etapa_caso', 'pais_hecho', 'ley', 'seccional','a_o_entrada', 'a_o_denuncia', 'total_procesos', 'consumado' ])
crime_news.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2889896 entries, 0 to 2890486
Data columns (total 6 columns):
 #   Column              Dtype 
---  ------              ----- 
 0   criminalidad        object
 1   departamento_hecho  object
 2   municipio_hecho     object
 3   a_o_hechos          int64 
 4   delito              object
 5   grupo_delito        object
dtypes: int64(1), object(5)
memory usage: 154.3+ MB


### Convertir columna a_o_hechos a tipo date

In [6]:
# Convertir la columna 'a_o_hechos' a tipo datetime
crime_news['a_o_hechos'] = pd.to_datetime(crime_news['a_o_hechos'], format='%Y')
crime_news['a_o_hechos'] = crime_news['a_o_hechos'].dt.year

### Verificar valores nulos

In [7]:
crime_news.isnull().sum()

,0
criminalidad,0
departamento_hecho,0
municipio_hecho,0
a_o_hechos,0
delito,0
grupo_delito,0


### Estandarización de categorizaciones

La estandarización de categorizaciones es el proceso de uniformizar y normalizar los valores de las categorías en un conjunto de datos para asegurar la consistencia y evitar discrepancias. Esto es crucial para la calidad y precisión de los análisis

In [8]:
# Imprimir categorías únicas para columnas de tipo object
for column in crime_news.select_dtypes(include='object').columns:
    print(f"Categorías en la columna '{column}':")
    print(crime_news[column].unique())
    print()

Categorías en la columna 'criminalidad':
['SI' 'NO']

Categorías en la columna 'departamento_hecho':
['Magdalena' 'Antioquia' 'Cundinamarca' 'BOGOTÁ, D. C.' 'Cesar'
 'Atlántico' 'Sucre' 'La Guajira' 'Cauca' 'Meta' 'Norte de Santander'
 'Valle del Cauca' 'Casanare' 'Santander' 'Chocó' 'Boyaca' 'Nariño'
 'Huila' 'Córdoba' 'Tolima' 'Risaralda' 'Caldas' 'Bolívar' 'Guaviare'
 'Quindío' 'Putumayo' 'Caquetá'
 'Archipiélago de San Andrés, Providencia y Santa Catalina' 'Arauca'
 'Amazonas' 'Guainía' 'Vichada' 'Vaupés']

Categorías en la columna 'municipio_hecho':
['SANTA MARTA' 'MEDELLÍN' 'FUSAGASUGÁ' ... 'PAPUNAUA' 'LA GUADALUPE'
 'PANA PANA']

Categorías en la columna 'delito':
['HOMICIDIO ART. 103 C.P.' 'ESTAFA. ART. 246 C.P. MENOR CUANTIA'
 'LESIONES PERSONALES CON INCAPACIDAD MENOR 60 DIAS' ...
 'MODALIDAD CULPOSA DEL FAVORECIMIENTO DE LA FUGA ART. 450 C.P. MODIF. ART. 10 LEY 733 DE 2002  INCISO 1'
 'OFENSA A DIPLOMATICOS ART. 466 C.P.'
 'ABORTO SIN CONSENTIMIENTO ART. 123 C.P. ATENUADO PO

  - Borrar espacios en blanco al principio y al final, cambiar a mayúsculas, remover acentos y eliminar signos extraños

In [9]:
import unicodedata

def remove_accents_and_special_chars(input_str):
    # Normalizar la cadena a NFKD
    nfkd_form = unicodedata.normalize('NFKD', input_str)

    # Eliminar acentos
    no_accents = ''.join([c for c in nfkd_form if not unicodedata.combining(c)])

    # Definir caracteres no deseados
    unwanted_chars = [',', ';', '!', '?', '#', '$', '%']

    # Eliminar caracteres no deseados
    cleaned_str = ''.join([c for c in no_accents if c not in unwanted_chars])

    # Remover espacios en blanco al principio y al final, y convertir a mayúsculas
    result = cleaned_str.strip().upper()

    return result

In [10]:
# Aplicar la función a todas las columnas categóricas
categorical_columns = ['criminalidad', 'departamento_hecho', 'municipio_hecho', 'delito', 'grupo_delito']

for col in categorical_columns:
    crime_news[col] = crime_news[col].apply(remove_accents_and_special_chars)

 - Codificación de algunas variables categóricas

In [11]:
import numpy as np

# Reemplazar valores en la columna 'criminalidad'
crime_news['criminalidad'] = crime_news['criminalidad'].replace({'SI': 1, 'NO': 0})

In [12]:
crime_news.head()

,criminalidad,departamento_hecho,municipio_hecho,a_o_hechos,delito,grupo_delito
0,1,MAGDALENA,SANTA MARTA,2013,HOMICIDIO ART. 103 C.P.,HOMICIDIO DOLOSO
1,0,ANTIOQUIA,MEDELLIN,2021,ESTAFA. ART. 246 C.P. MENOR CUANTIA,ESTAFA
2,1,CUNDINAMARCA,FUSAGASUGA,2014,LESIONES PERSONALES CON INCAPACIDAD MENOR 60 DIAS,LESIONES PERSONALES
3,0,BOGOTA D. C.,BOGOTA D.C.,2023,VIOLACION DE DATOS PERSONALES ART 269F LEY 127...,DELITOS INFORMATICOS
4,1,CESAR,VALLEDUPAR,2017,HURTO. ART. 239 C.P.,HURTO


## Agregar código del municipio a dataframe crime_news para facilitar la georeferenciación

### -  Cargar los datos con códigos de departamentos y municipios

Como producto de una consulta a la base de datos del proyecto que se esta construyendo (Tablas departments y municipalities) se creo el archivo csv que se carga en la siguiente celda, y que incluye los nombres de los departamentos y municipios con sus respectivos codigos, generados por el DANE

In [13]:
dept_mpios_codes = pd.read_csv("/content/drive/MyDrive/analytics_data_proyect/deptos_mupios.csv", index_col=0, dtype={'dept_mpio_code': str})
print(dept_mpios_codes.info())
dept_mpios_codes.head()

<class 'pandas.core.frame.DataFrame'>
Index: 1121 entries, 0 to 1120
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   dept_mpio_code  1121 non-null   object
 1   dept_name       1121 non-null   object
 2   mupio_name      1121 non-null   object
dtypes: object(3)
memory usage: 35.0+ KB
None


,dept_mpio_code,dept_name,mupio_name
0,97001,VAUPES,MITU
1,97161,VAUPES,CARURU
2,97511,VAUPES,PACOA
3,97666,VAUPES,TARAIRA
4,97777,VAUPES,PAPUNAHUA


### -  Asegurarse de que los nombres de los departamentos estén escritos de manera identica en cada datafram

  - Borrar espacios en blanco al principio y al final, cambiar a mayúsculas, remover acentos y eliminar signos extraños

In [14]:
# Aplicar funcion a columnas 'dept_name', 'mupio_name'
for col in dept_mpios_codes[['dept_name', 'mupio_name']]:
    dept_mpios_codes[col] = dept_mpios_codes[col].apply(remove_accents_and_special_chars)

  - Comparar que los nombres de los departamentos y municipios en cada dataframe esten escritos correctamente

In [15]:
# Función para comparar listas y mostrar diferencias
def compare_lists(df1_col, df2_col, label1, label2):
    # Extraer listas únicas y normalizar
    list1 = set(df1_col.str.strip().str.upper().unique())
    list2 = set(df2_col.str.strip().str.upper().unique())

    # Encontrar diferencias
    only_in_list1 = list1 - list2
    only_in_list2 = list2 - list1

    # Imprimir resultados
    print(f"{label1} que no están en {label2}:")
    print(only_in_list1)
    print(f"{label2} que no están en {label1}:")
    print(only_in_list2)
    print()


  - Comparar nombres de departamentos en los dataframes

In [16]:
# Comparar listas de departamentos
compare_lists(crime_news['departamento_hecho'], dept_mpios_codes['dept_name'],
              "Departamentos en crime_news", "Departamentos dept_mpios_codes")

Departamentos en crime_news que no están en Departamentos dept_mpios_codes:
{'BOGOTA D. C.'}
Departamentos dept_mpios_codes que no están en Departamentos en crime_news:
{'BOGOTA D.C.'}



  - Solucionar discrepancias en los nombres de departamentos

In [17]:
# Diccionario de mapeo basado en los resultados de la comparación departamentos
depto_mapping = { 'BOGOTA D. C.': 'BOGOTA D.C.'}

# Reemplazar los nombres incorrectos  el dataframe crime_news
crime_news['departamento_hecho'] = crime_news['departamento_hecho'].replace(depto_mapping)

  - Confirmar concordancia nombres departamentos

In [18]:
# Comparar listas de departamento
compare_lists(crime_news['departamento_hecho'], dept_mpios_codes['dept_name'],
              "Departamentos en crime_news", "Departamentos dept_mpios_codes")

Departamentos en crime_news que no están en Departamentos dept_mpios_codes:
set()
Departamentos dept_mpios_codes que no están en Departamentos en crime_news:
set()



  - Comparar nombres de los departamentos combinados con los nombres de los municipios en los dos dataframe

In [19]:
# Crear columnas combinadas de departamento + municipio
crime_news['departamento_municipio'] = crime_news['departamento_hecho'] + ' - ' + crime_news['municipio_hecho']
dept_mpios_codes['departamento_municipio'] = dept_mpios_codes['dept_name'] + ' - ' + dept_mpios_codes['mupio_name']

# Comparar listas combinadas de departamento + municipio
compare_lists(crime_news['departamento_municipio'], dept_mpios_codes['departamento_municipio'],
              "Departamentos y Municipios en crime_news", "Departamentos y Municipios en dept_mpios_codes")

Departamentos y Municipios en crime_news que no están en Departamentos y Municipios en dept_mpios_codes:
{'ANTIOQUIA - DON MATIAS', 'CORDOBA - PURISIMA', 'CAUCA - EL BORDO', 'CESAR - MANAURE', 'PUTUMAYO - LEGUIZAMO', 'GUAINIA - BARRANCO MINAS', 'VAUPES - PAPUNAUA', 'CUNDINAMARCA - SAN JUAN DE RIO SECO', 'NORTE DE SANTANDER - CUCUTA', 'CAUCA - SOTARA', 'BOLIVAR - MOMPOS', 'CORDOBA - SAN ANDRES SOTAVENTO', 'CHOCO - BELEN DE BAJIRA', 'TOLIMA - MARIQUITA', 'CAUCA - LOPEZ', 'GUAINIA - MAPIRIPANA', 'BOLIVAR - CARTAGENA', 'NARINO - TUMACO', 'MAGDALENA - CHIBOLO', 'NARINO - CUASPUD', 'BOYACA - GUICAN', 'ANTIOQUIA - SANTAFE DE ANTIOQUIA', 'ANTIOQUIA - SAN VICENTE', 'ANTIOQUIA - SAN PEDRO', 'SUCRE - TOLU VIEJO', 'MAGDALENA - CERRO SAN ANTONIO', 'CUNDINAMARCA - UBATE', 'CAUCA - PIENDAMO'}
Departamentos y Municipios en dept_mpios_codes que no están en Departamentos y Municipios en crime_news:
{'SUCRE - SAN JOSE DE TOLUVIEJO', 'ANTIOQUIA - DONMATIAS', 'CAUCA - PIENDAMO - TUNIA', 'PUTUMAYO - PUERTO 

 - Corregir nombres de los departamentos combinados con los nombres en dataframe crime_news

In [20]:
mapeo_manual = {
    'CUNDINAMARCA - SAN JUAN DE RIO SECO': 'CUNDINAMARCA - SAN JUAN DE RIOSECO',
    'TOLIMA - MARIQUITA': 'TOLIMA - SAN SEBASTIAN DE MARIQUITA',
    'CORDOBA - PURISIMA': 'CORDOBA - PURISIMA DE LA CONCEPCION',
    'BOLIVAR - MOMPOS': 'BOLIVAR - SANTA CRUZ DE MOMPOX',
    'NORTE DE SANTANDER - CUCUTA': 'NORTE DE SANTANDER - SAN JOSE DE CUCUTA',
    'CAUCA - EL BORDO': 'CAUCA - PATIA',
    'NARINO - TUMACO': 'NARINO - SAN ANDRES DE TUMACO',
    'CAUCA - SOTARA': 'CAUCA - SOTARA PAISPAMBA',
    'ANTIOQUIA - SAN PEDRO': 'ANTIOQUIA - SAN PEDRO DE LOS MILAGROS',
    'ANTIOQUIA - DON MATIAS': 'ANTIOQUIA - DONMATIAS',
    'CHOCO - BELEN DE BAJIRA': 'ANTIOQUIA - MUTATA',
    'BOLIVAR - CARTAGENA': 'BOLIVAR - CARTAGENA DE INDIAS',
    'NARINO - CUASPUD': 'NARINO - CUASPUD CARLOSAMA',
    'CORDOBA - SAN ANDRES SOTAVENTO': 'CORDOBA - SAN ANDRES DE SOTAVENTO',
    'MAGDALENA - CERRO SAN ANTONIO': 'MAGDALENA - CERRO DE SAN ANTONIO',
    'BOYACA - GUICAN': 'BOYACA - GUICAN DE LA SIERRA',
    'CUNDINAMARCA - UBATE': 'CUNDINAMARCA - VILLA DE SAN DIEGO DE UBATE',
    'CAUCA - LOPEZ': 'CAUCA - LOPEZ DE MICAY',
    'CESAR - MANAURE': 'CESAR - MANAURE BALCON DEL CESAR',
    'CAUCA - PIENDAMO': 'CAUCA - PIENDAMO - TUNIA',
    'GUAINIA - BARRANCO MINAS': 'GUAINIA - BARRANCOMINAS',
    'MAGDALENA - CHIBOLO': 'MAGDALENA - CHIVOLO',
    'SUCRE - TOLU VIEJO': 'SUCRE - SAN JOSE DE TOLUVIEJO',
    'GUAINIA - MAPIRIPANA': 'GUAINIA - PANA PANA',
    'ANTIOQUIA - SANTAFE DE ANTIOQUIA': 'ANTIOQUIA - SANTA FE DE ANTIOQUIA',
    'ANTIOQUIA - SAN VICENTE': 'ANTIOQUIA - SAN VICENTE FERRER',
    'VAUPES - PAPUNAUA': 'VAUPES - PAPUNAHUA',
    'PUTUMAYO - LEGUIZAMO': 'PUTUMAYO - PUERTO LEGUIZAMO',

}


crime_news['departamento_municipio'] = crime_news['departamento_municipio'].replace(mapeo_manual)

 - Comparar nombres de los departamentos combinados con los nombres de los municipios despues de corregir

In [21]:
# Comparar listas combinadas de departamento + municipio
compare_lists(crime_news['departamento_municipio'], dept_mpios_codes['departamento_municipio'],
              "Departamentos y Municipios en crime_news", "Departamentos y Municipios en dept_mpios_codes")

Departamentos y Municipios en crime_news que no están en Departamentos y Municipios en dept_mpios_codes:
set()
Departamentos y Municipios en dept_mpios_codes que no están en Departamentos y Municipios en crime_news:
set()



### Hacer merge entre los dataframes crime_news y dept_mpios_codes

In [22]:
# Realizamos el merge para obtener el código de municipio
crime_news_with_codes = crime_news.merge(dept_mpios_codes,
                                         left_on=['departamento_municipio'],
                                         right_on=['departamento_municipio'],
                                         how='left')

# Verificamos el resultado
crime_news_with_codes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2889896 entries, 0 to 2889895
Data columns (total 10 columns):
 #   Column                  Dtype 
---  ------                  ----- 
 0   criminalidad            int64 
 1   departamento_hecho      object
 2   municipio_hecho         object
 3   a_o_hechos              int32 
 4   delito                  object
 5   grupo_delito            object
 6   departamento_municipio  object
 7   dept_mpio_code          object
 8   dept_name               object
 9   mupio_name              object
dtypes: int32(1), int64(1), object(8)
memory usage: 209.5+ MB


In [23]:
crime_news_with_codes.head()

,criminalidad,departamento_hecho,municipio_hecho,a_o_hechos,delito,grupo_delito,departamento_municipio,dept_mpio_code,dept_name,mupio_name
0,1,MAGDALENA,SANTA MARTA,2013,HOMICIDIO ART. 103 C.P.,HOMICIDIO DOLOSO,MAGDALENA - SANTA MARTA,47001,MAGDALENA,SANTA MARTA
1,0,ANTIOQUIA,MEDELLIN,2021,ESTAFA. ART. 246 C.P. MENOR CUANTIA,ESTAFA,ANTIOQUIA - MEDELLIN,05001,ANTIOQUIA,MEDELLIN
2,1,CUNDINAMARCA,FUSAGASUGA,2014,LESIONES PERSONALES CON INCAPACIDAD MENOR 60 DIAS,LESIONES PERSONALES,CUNDINAMARCA - FUSAGASUGA,25290,CUNDINAMARCA,FUSAGASUGA
3,0,BOGOTA D.C.,BOGOTA D.C.,2023,VIOLACION DE DATOS PERSONALES ART 269F LEY 127...,DELITOS INFORMATICOS,BOGOTA D.C. - BOGOTA D.C.,11001,BOGOTA D.C.,BOGOTA D.C.
4,1,CESAR,VALLEDUPAR,2017,HURTO. ART. 239 C.P.,HURTO,CESAR - VALLEDUPAR,20001,CESAR,VALLEDUPAR


In [24]:
# No de filas por columna que no encontraron coincidencia
print(crime_news_with_codes.isnull().sum())

criminalidad              0
departamento_hecho        0
municipio_hecho           0
a_o_hechos                0
delito                    0
grupo_delito              0
departamento_municipio    0
dept_mpio_code            0
dept_name                 0
mupio_name                0
dtype: int64


### Procesamiento final como preparación para integrarlo a la bd de datos del proyecto

In [25]:
# Eliminar columnas innecesarias
columns_to_drop = ['departamento_hecho', 'municipio_hecho', 'departamento_municipio', 'dept_name', 'mupio_name']
final_crime_news = crime_news_with_codes.drop(columns=columns_to_drop)

In [26]:
# Adicionar columna para trazabilidad de la fuente
final_crime_news['source_id'] = 3

In [27]:
# Ajustar nombre de columnas

# Definir el diccionario de traducción
translation_map = {
    'criminalidad': 'crime_severity',
    'a_o_hechos': 'year_of_incident',
    'delito': 'crime_type',
    'grupo_delito': 'crime_group',
    'dept_mpio_code': 'dept_mpio_code',
    'source_id': 'source_id'
}

# Renombrar las columnas
final_crime_news.rename(columns=translation_map, inplace=True)

In [28]:
#Estructura final del dataset a integrar a la base de datos
final_crime_news.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2889896 entries, 0 to 2889895
Data columns (total 6 columns):
 #   Column            Dtype 
---  ------            ----- 
 0   crime_severity    int64 
 1   year_of_incident  int32 
 2   crime_type        object
 3   crime_group       object
 4   dept_mpio_code    object
 5   source_id         int64 
dtypes: int32(1), int64(2), object(3)
memory usage: 121.3+ MB


### Salvar en archivo csv en el drive

In [ ]:
# Guardar en archivos CSV en el drive
final_crime_news.to_csv('/content/drive/MyDrive/analytics_data_proyect/initial_transformation/crime_news.csv', index=False)